In [ ]:
import numpy as np
import pandas as pd

class Die:
    def __init__(self, faces):
        """
        Initializes a die with given faces.
        
        Args:
            faces (numpy array): Array of faces for the die, can be strings or numbers.
        
        Raises:
            TypeError: If faces is not a numpy array.
            ValueError: If faces do not contain distinct values.
        """
        if not isinstance(faces, np.ndarray):
            raise TypeError("Faces must be a NumPy array.")
        if len(faces) != len(set(faces)):
            raise ValueError("Faces must be distinct values.")
        
        self._faces = faces
        self._weights = np.ones(len(faces))  # Default weight of 1.0 for each face
        self._die_df = pd.DataFrame({
            'face': faces,
            'weight': self._weights
        }).set_index('face')

    def change_weight(self, face, new_weight):
        """
        Changes the weight of a given face.
        
        Args:
            face: The face whose weight needs to be changed.
            new_weight (float): The new weight for the given face.
        
        Raises:
            IndexError: If the face is not in the die.
            TypeError: If the new weight is not numeric.
        """
        if face not in self._die_df.index:
            raise IndexError("Face value not found in die.")
        if not isinstance(new_weight, (int, float)) or new_weight < 0:
            raise TypeError("Weight must be a positive numeric value.")
        
        self._die_df.at[face, 'weight'] = float(new_weight)

    def roll(self, rolls=1):
        """
        Rolls the die a given number of times.
        
        Args:
            rolls (int): Number of times the die should be rolled. Defaults to 1.
        
        Returns:
            list: List of outcomes from rolling the die.
        """
        return self._die_df.sample(n=rolls, weights='weight', replace=True).index.tolist()
    
    def show(self):
        """
        Returns the current state of the die.
        
        Returns:
            pandas.DataFrame: A copy of the current die with faces and their weights.
        """
        return self._die_df.copy()


class Game:
    def __init__(self, dice):
        """
        Initializes the game with a list of similar Die objects.
        
        Args:
            dice (list): List of Die objects.
        
        Raises:
            TypeError: If dice is not a list of Die objects.
        """
        if not isinstance(dice, list) or not all(isinstance(die, Die) for die in dice):
            raise TypeError("All elements must be Die objects.")
        self.dice = dice
        self._results = None

    def play(self, rolls=1):
        """
        Rolls all dice a given number of times and records the results.
        
        Args:
            rolls (int): Number of times to roll the dice.
        """
        results = {f"Die_{i}": die.roll(rolls) for i, die in enumerate(self.dice)}
        self._results = pd.DataFrame(results)
        self._results.index.name = 'Roll'

    def show(self, form='wide'):
        """
        Shows the results of the most recent play.
        
        Args:
            form (str): The format to return the results in ('wide' or 'narrow'). Defaults to 'wide'.
        
        Returns:
            pandas.DataFrame: DataFrame of results in the specified format.
        
        Raises:
            ValueError: If an invalid format is passed.
        """
        if self._results is None:
            raise ValueError("No results available. Please play the game first.")
        if form == 'wide':
            return self._results
        elif form == 'narrow':
            return self._results.stack().to_frame('Outcome')
        else:
            raise ValueError("Invalid form. Use 'wide' or 'narrow'.")


class Analyzer:
    def __init__(self, game):
        """
        Initializes the analyzer with a game object.
        
        Args:
            game (Game): A Game object whose results are to be analyzed.
        
        Raises:
            TypeError: If the input is not a Game object.
        """
        if not isinstance(game, Game):
            raise TypeError("Input must be a Game object.")
        self.game = game
        self.results = game.show(form='wide')

    def jackpot(self):
        """
        Computes the number of times all dice show the same face.
        
        Returns:
            int: The number of jackpots (i.e., rolls where all dice have the same face).
        """
        return (self.results.nunique(axis=1) == 1).sum()

    def face_counts_per_roll(self):
        """
        Computes the counts of each face per roll.
        
        Returns:
            pandas.DataFrame: DataFrame with the count of each face value per roll.
        """
        return self.results.apply(pd.Series.value_counts, axis=1).fillna(0)

    def combo_count(self):
        """
        Computes the distinct combinations of faces rolled, order-independent.
        
        Returns:
            pandas.DataFrame: DataFrame with combinations as index and their counts.
        """
        sorted_rolls = self.results.apply(lambda x: tuple(sorted(x)), axis=1)
        return sorted_rolls.value_counts().to_frame('Count')

    def permutation_count(self):
        """
        Computes the distinct permutations of faces rolled, order-dependent.
        
        Returns:
            pandas.DataFrame: DataFrame with permutations as index and their counts.
        """
        perm_rolls = self.results.apply(lambda x: tuple(x), axis=1)
        return perm_rolls.value_counts().to_frame('Count')

    def scrabble_word_analysis(self, word_list):
        """
        Analyzes the results to determine if any rolls form valid Scrabble words.
        
        Args:
            word_list (list): A list of valid Scrabble words.
        
        Returns:
            pandas.DataFrame: DataFrame indicating which rolls form valid Scrabble words.
        """
        valid_words = []
        for roll in self.results.apply(lambda x: ''.join(sorted(x)), axis=1):
            if roll in word_list:
                valid_words.append(True)
            else:
                valid_words.append(False)
        
        return pd.DataFrame({'Roll': self.results.index, 'Is_Valid_Word': valid_words})

    def letter_frequency_analysis(self, letter_frequencies):
        """
        Computes the frequency of each letter rolled compared to expected frequencies in the English language.
        
        Args:
            letter_frequencies (dict): A dictionary of letters and their frequencies in the English language.
        
        Returns:
            pandas.DataFrame: DataFrame comparing rolled letter frequencies to expected frequencies.
        """
        rolled_letter_counts = self.results.apply(pd.Series.value_counts).sum().fillna(0)
        comparison = pd.DataFrame({'Rolled_Count': rolled_letter_counts})
        comparison['Expected_Frequency'] = comparison.index.map(letter_frequencies)
        comparison['Expected_Frequency'] = comparison['Expected_Frequency'].fillna(0)
        return comparison

# Load external data for Scrabble words and English letter frequencies
with open('scrabble_words.txt') as f:
    scrabble_words = [word.strip().upper() for word in f.readlines()]

letter_frequencies = {}
with open('english_letters.txt') as f:
    for line in f:
        letter, frequency = line.split()
        letter_frequencies[letter] = int(frequency)

# Now the scrabble_words and letter_frequencies can be passed to Analyzer methods


In [ ]:
# Creating Die objects using the faces from the English alphabet
faces = np.array(list("ABCDEFGHIJKLMNOPQRSTUVWXYZ"))
die1 = Die(faces)
die2 = Die(faces)

# Creating a Game with two dice
game = Game([die1, die2])

# Playing the game with 10 rolls
game.play(rolls=10)

# Showing the game results in wide format
print("Game Results (Wide Format):")
print(game.show(form='wide'))

# Analyzing the game results
analyzer = Analyzer(game)

# Computing the number of jackpots
print("Number of Jackpots:", analyzer.jackpot())

# Face counts per roll
print("Face Counts per Roll:")
print(analyzer.face_counts_per_roll())

# Combination count (order-independent)
print("Combination Count:")
print(analyzer.combo_count())

# Permutation count (order-dependent)
print("Permutation Count:")
print(analyzer.permutation_count())

# Scrabble word analysis
print("Scrabble Word Analysis:")
print(analyzer.scrabble_word_analysis(scrabble_words))

# Letter frequency analysis
print("Letter Frequency Analysis:")
print(analyzer.letter_frequency_analysis(letter_frequencies))


In [ ]:
import numpy as np
from __main__ import Die, Game, Analyzer
import pandas as pd

# Scenario 1: Fair vs. Unfair Dice Simulation
faces = np.array([1, 2, 3, 4, 5, 6])

# Fair die (all weights equal)
fair_die = Die(faces)

# Unfair die (higher weight for face '6')
unfair_die = Die(faces)
unfair_die.change_weight(6, 5.0)

# Play games with fair and unfair dice
fair_game = Game([fair_die])
unfair_game = Game([unfair_die])

fair_game.play(rolls=1000)
unfair_game.play(rolls=1000)

# Analyze results
fair_analyzer = Analyzer(fair_game)
unfair_analyzer = Analyzer(unfair_game)

print("Fair Die - Face Counts Per Roll:")
print(fair_analyzer.face_counts_per_roll().sum())

print("Unfair Die - Face Counts Per Roll:")
print(unfair_analyzer.face_counts_per_roll().sum())

# Scenario 2: Jackpot Analysis in a Dice Game
die1 = Die(np.array(['A', 'B', 'C']))
die2 = Die(np.array(['A', 'B', 'C']))
die3 = Die(np.array(['A', 'B', 'C']))

game = Game([die1, die2, die3])
game.play(rolls=1000)

analyzer = Analyzer(game)

print("Number of Jackpots in the Game:")
print(analyzer.jackpot())

# Scenario 3: Simulating Letter Frequency and Valid Scrabble Words
letters = np.array(list("ABCDEFGHIJKLMNOPQRSTUVWXYZ"))
die1 = Die(letters)
die2 = Die(letters)
die3 = Die(letters)
die4 = Die(letters)
die5 = Die(letters)

game = Game([die1, die2, die3, die4, die5])
game.play(rolls=500)

analyzer = Analyzer(game)

# Scrabble word analysis
with open('scrabble_words.txt') as f:
    scrabble_words = [word.strip().upper() for word in f.readlines()]

print("Scrabble Word Analysis:")
print(analyzer.scrabble_word_analysis(scrabble_words))

# Letter frequency analysis
letter_frequencies = {
    'A': 8.17, 'B': 1.49, 'C': 2.78, 'D': 4.25, 'E': 12.70, 'F': 2.23, 'G': 2.02,
    'H': 6.09, 'I': 6.97, 'J': 0.15, 'K': 0.77, 'L': 4.03, 'M': 2.41, 'N': 6.75,
    'O': 7.51, 'P': 1.93, 'Q': 0.10, 'R': 5.99, 'S': 6.33, 'T': 9.06, 'U': 2.76,
    'V': 0.98, 'W': 2.36, 'X': 0.15, 'Y': 1.97, 'Z': 0.07
}

print("Letter Frequency Analysis:")
rolled_letter_counts = analyzer.face_counts_per_roll().sum()
comparison = pd.DataFrame({'Rolled_Count': rolled_letter_counts})
comparison['Expected_Frequency'] = comparison.index.map(letter_frequencies)
comparison['Expected_Frequency'] = comparison['Expected_Frequency'].fillna(0)
print(comparison)

